# 반복 단어, 문장 1개로 통일

In [73]:
!pip install rouge

  Obtaining dependency information for rouge from https://files.pythonhosted.org/packages/32/7c/650ae86f92460e9e8ef969cc5008b24798dcf56a9a8947d04c78f550b3f5/rouge-1.0.1-py3-none-any.whl.metadata


In [71]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
import rouge
from collections import defaultdict
from tqdm import tqdm

In [73]:
def make_dataframe(path: str) -> pd.DataFrame:
    """
    Read a json file and return a pandas DataFrame.

    Parameters:
    path (str): Path to the json file.

    Returns:
    pd.DataFrame: DataFrame of the json file.
    """
    # Read the json file
    with open(path, 'r') as file:
        data = json.load(file)

    # Create a DataFrame
    # columns = ['id', 'conversation', 'subject_keyword', 'output']
    df = pd.DataFrame(data)
    df['conversation'] = df['input'].apply(lambda x: x['conversation'])
    df['subject_keyword'] = df['input'].apply(lambda x: x['subject_keyword'])

    # Drop the 'input' column
    df.drop('input', axis=1, inplace=True)

    # Speakers in the conversation
    df['speakers'] = df['conversation'].apply(lambda turns: list(set(turn['speaker'] for turn in turns)))

    # Reorder the columns
    df = df[['id', 'conversation', 'subject_keyword', 'speakers', 'output']]

    return df

In [4]:
train_df = make_dataframe('../resource/data/일상대화요약_train.json')
dev_df = make_dataframe('../resource/data/일상대화요약_dev.json')
test_df = make_dataframe('../resource/data/일상대화요약_test.json')
filtered_train_df = make_dataframe('./train.json')
filtered_dev_df = make_dataframe('./dev.json')
filtered_test_df = make_dataframe('./test.json')

In [76]:
hypernova = make_dataframe('../results/hypernova.json')
cosmos = make_dataframe('../results/cosmos.json')
cosmos2 = make_dataframe('../results/cosmos2.json')
cosmos25 = make_dataframe('../results/cosmos_25.json')
cosmosre12 = make_dataframe('../results/cosmos_re12.json')

## output 내의 반복되는 단어

In [6]:
# output 내에 '\b([가-힣a-zA-Z0-9_]+)\s+\1\b'를 만족하는 부분이 있는지 확인하고 있다면 set에 추가하고 마지막에 return하는 함수

def find_repeated_words(df: pd.DataFrame, pattern) -> set:
    """
    Find repeated words in the 'output' column of the DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame to search for repeated words.
    pattern (str): Pattern to search for.

    Returns:
    set: Set of repeated words.
    """
    count = 0

    repeated_words = set()
    for output in df['output']:
        matches = re.findall(pattern, output)
        repeated_words.update(matches)
        count+=len(matches)

    print(f"Total number of repeated words found: {len(repeated_words)}")
    print(f"Total number of repeated words found: {count}")

    return repeated_words


In [6]:
repeated_words = find_repeated_words(filtered_train_df, r'\b(\w+)\b(?:\s+\1\b)+')
repeated_words

Total number of repeated words found: 8
Total number of repeated words found: 12


{'가도', '그리고', '깜짝', '대화에서', '두루', '좋은', '할머니가', '화자'}

In [7]:
repeated_words = find_repeated_words(filtered_dev_df, r'\b(\w+)\s+\1\b')

Total number of repeated words found: 0
Total number of repeated words found: 0


In [53]:
repeated_words = find_repeated_words(cosmos25, r'\b(\w+)\s+\1\b')
repeated_words

Total number of repeated words found: 0
Total number of repeated words found: 0


set()

In [14]:
# 전처리 이후
filtered_train_df = make_dataframe('./train.json')
filtered_dev_df = make_dataframe('./dev.json')
filtered_test_df = make_dataframe('./test.json')

repeated_words = find_repeated_words(filtered_train_df, r'\b(\w+)\b(?:\s+\1\b)+')
repeated_words

Total number of repeated words found: 2
Total number of repeated words found: 2


{'깜짝', '두루'}

## output 내의 반복되는 문장

In [68]:
# output 내의 반복되는 문장 찾기
rouge_score = rouge.Rouge().get_scores

def find_repeated_sentence(df: pd.DataFrame) -> set:
    """
    Find repeated sentences in the DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame of the output.

    Returns:
    set: Set of repeated sentences.
    """
    exact_repeated_sentences = set()
    high_similarity_repeated_sentences = defaultdict(list)
    exact_cnt = 0
    high_similarity_cnt = 0
    exact_idxes = set()
    high_similarity_idxes = set()


    for idx, output in tqdm(enumerate(df['output']), total=len(df)):
        output_sentences = output.split('.')[:-1]

        # exact_repeated_sentences
        for sentence in output_sentences: # 마지막은 빈 문자열이므로 제외
            if output.count(sentence) > 1:
                exact_repeated_sentences.add(sentence)
                output_sentences.remove(sentence)
                exact_cnt+=1
                exact_idxes.add(idx)

        # high_similarity_repeated_sentences
        for sent_idx in range(1,len(output_sentences)-1):
            for unique_sentence in output_sentences[:sent_idx]:
                scores = rouge_score(output_sentences[sent_idx], unique_sentence)
                if scores[0]['rouge-1']['f'] > 0.6:
                    high_similarity_repeated_sentences[unique_sentence].append((idx, unique_sentence, output_sentences[sent_idx]))
                    high_similarity_cnt+=1
                    high_similarity_idxes.add(idx)

    print(f"Total number of exact repeated sentences found: {exact_cnt}")
    print(f"the number of exact repeated sentences found: {len(exact_repeated_sentences)}")
    print(f"Total number of high similarity repeated sentences found: {high_similarity_cnt}")
    print(f"the number of high similarity repeated sentences found: {len(high_similarity_repeated_sentences)}")
    print(f"Exact repeated sentences found in the following indexes: {exact_idxes}")
    print(f"High similarity repeated sentences found in the following indexes: {high_similarity_idxes}")

    return exact_repeated_sentences, high_similarity_repeated_sentences, exact_idxes, high_similarity_idxes

In [40]:
repeated_sentences, hs, _, _ = find_repeated_sentence(filtered_train_df)

Total number of exact repeated sentences found: 0
the number of exact repeated sentences found: 0
Total number of high similarity repeated sentences found: 0
the number of high similarity repeated sentences found: 0
Exact repeated sentences found in the following indexes: set()
High similarity repeated sentences found in the following indexes: set()


In [29]:
repeated_sentences = find_repeated_sentence(filtered_dev_df)

Total number of exact repeated sentences found: 1
the number of exact repeated sentences found: 1
Total number of high similarity repeated sentences found: 1
the number of high similarity repeated sentences found: 1
Exact repeated sentences found in the following indexes: [92]
High similarity repeated sentences found in the following indexes: [92]


In [30]:
repeated_sentences

({'SD2000862는 엔시티의 보스 착장 중 제복 착장이 멤버들의 피지컬 덕분에 좋았고 터프하게 랩을 하는 마크의 모습이 인상깊었다고 말했습니다'},
 {('SD2000862는 엔시티의 보스 착장 중 제복 착장이 멤버들의 피지컬 덕분에 좋았고 터프하게 랩을 하는 마크의 모습이 인상깊었다고 말했습니다',
   ' SD2000862는 엔시티의 보스 착장 중 제복 착장이 멤버들의 피지컬 덕분에 좋았고 터프하게 랩을 하는 마크의 모습이 인상깊었다고 말했습니다')})

- 중복 제거 이후

In [37]:
# 전처리 이후
filtered_train_df = make_dataframe('./train.json')
filtered_dev_df = make_dataframe('./dev.json')
filtered_test_df = make_dataframe('./test.json')
repeated_sentences, hs = find_repeated_sentence(filtered_dev_df)

Total number of exact repeated sentences found: 0
the number of exact repeated sentences found: 0
Total number of high similarity repeated sentences found: 0
the number of high similarity repeated sentences found: 0
Exact repeated sentences found in the following indexes: set()
High similarity repeated sentences found in the following indexes: set()


### result 분석

In [69]:
repeated_sentences, high_similarity_repeated_sentences, exact_idxes, high_idxes = find_repeated_sentence(cosmos25) # cosmosre12

  0%|          | 0/408 [00:00<?, ?it/s]

100%|██████████| 408/408 [00:00<00:00, 855.35it/s]

Total number of exact repeated sentences found: 22
the number of exact repeated sentences found: 9
Total number of high similarity repeated sentences found: 70
the number of high similarity repeated sentences found: 35
Exact repeated sentences found in the following indexes: {166, 328, 269, 23, 216, 219, 221}
High similarity repeated sentences found in the following indexes: {129, 130, 261, 137, 396, 14, 15, 149, 26, 282, 166, 169, 302, 310, 316, 68, 328, 204, 206, 336, 219, 349, 98, 356, 103, 360, 105, 111, 369, 125}


In [74]:
exact_idxes in high_idxes

False

In [61]:
repeated_sentences

{' SD2000282는 SD2000283이 많이 먹는 것 같으니 차슈 덩밥을 먹고 싶어서 상무지구의 차슈 덩밥을 먹으러 가자고 말했습니다',
 ' SD2000282는 SD2000283이 많이 먹는 것 같으니 차슈 덮밥을 먹고 싶어서 상무지구의 차슈 덮밥을 먹으러 가자고 말했습니다',
 ' SD2000287은 중학교 1학년 때 동아리로 연극동아리에 가입하게 된 이유는 선생님께서 추천해주셨기 때문이라고 답했습니다',
 ' 단점으로는 감기와 추위가 심해 밖에서의 활동이 제한될 수 있다고 했습니다',
 ' 또 대전여중 앞쪽에 두부 두루치기가 있고, 성모병원 앞에도 두부 두루치기가 있다고 말했습니다',
 ' 또 자신이 선물을 선택할 때는 가성비보다는 자신이 좋아하는 것을 선물한다고 했습니다',
 ' 또 초등학교 때 가장 친했던 친구가 있었는지 물어봤으며, 중학교 때 연극동아리를 하면서 느낀 장점과 단점을 질문했습니다',
 ' 또한 혈액형은 성격과 관련이 없다는 것을 알고 있어서 혈액형에 의존하여 성격을 판단하지 않는다고 말했습니다',
 ' 여행 방식은 즉흥적인 여행을 선호한다고 말했습니다'}

In [70]:
for key in high_similarity_repeated_sentences.keys():
    idx = high_similarity_repeated_sentences[key][0][0]
    print(f"[{idx}][중복되는 문장중에서 가장 첫 번째로 등장하는 문장] :", key, '\n')
    for sentence in high_similarity_repeated_sentences[key]:
        print(' - ', sentence[2])
    print('\n\n')
    

[14][중복되는 문장중에서 가장 첫 번째로 등장하는 문장] :  SD2000058은 가족 중에서 가장 건강에 염려스러운 것은 치매라고 말했습니다 

 -   또 가족 중에서 가장 건강에 염려스러운 것은 치매라고 말했고, 가족 중에서 가장 건강에 염려스러운 것은 치매라고 말했습니다
 -   SD2000059는 가족 중에서 가장 건강에 염려스러운 것은 고혈압이라고 말했습니다



[14][중복되는 문장중에서 가장 첫 번째로 등장하는 문장] :  또 가족 중에서 가장 건강에 염려스러운 것은 치매라고 말했고, 가족 중에서 가장 건강에 염려스러운 것은 치매라고 말했습니다 

 -   SD2000059는 가족 중에서 가장 건강에 염려스러운 것은 고혈압이라고 말했습니다



[15][중복되는 문장중에서 가장 첫 번째로 등장하는 문장] :  SD2000059는 코로나에 걸렸을 때 극심한 근육통, 고열, 기침 등의 증상을 보일 수도 있으며 무증상 환자도 있을 수 있다고 말했습니다 

 -   또 코로나에 걸리면 근육통, 고열, 기침 등의 증상을 보일 수 있으며 무증상 환자도 있을 수 있다고 말했습니다



[26][중복되는 문장중에서 가장 첫 번째로 등장하는 문장] :  SD2000066은 남자 친구가 생긴다면 가장 먹고 싶은 음식은 이탈리아 음식이라고 말했습니다 

 -   SD2000067은 남자 친구가 생긴다면 가장 먹고 싶은 음식은 이탈리아 음식이라고 말했습니다



[68][중복되는 문장중에서 가장 첫 번째로 등장하는 문장] :  그리고 가족이 타인으로 대우받는다고 생각하며 가족과의 갈등을 해결하기 위해 타인을 기준으로 생각해보려고 한다며 가족과의 갈등을 해결하려는 방법을 설명했습니다 

 -   SD2000114는 가족이 타인으로 대우받는다고 생각하며 가족과의 갈등을 해결하기 위해 가족과의 갈등을 해결하려는 방법을 설명했습니다



[98][중복되는 문장중에서 가장 첫 번째로 등장하는 문장] :  또 중학교 때는 조퇴해 집에서 땡까땡까 놀았고, 고등학교 때는 친구

In [77]:
def compare_outputs(idx: int):
    """
    Compare the outputs of the different models.

    Parameters:
    idx (int): Index of the output to compare.

    Returns:
    None
    """
    print(f"Index: {idx}")
    print(f"Hypernova: {hypernova['output'][idx]}")
    print(f"Cosmos2: {cosmos2['output'][idx]}")
    print(f"Cosmos25: {cosmos25['output'][idx]}")

In [116]:
compare_outputs(365)

Index: 365
Hypernova: 이 대화에서 SD2000568과 SD2000569는 국내외 여행지에 대해 이야기했습니다. SD2000568은 국내 여행지로는 서울이나 자연경관을 볼 수 있는 한적한 곳을 가보고 싶다고 말했습니다. 해외 여행지로는 일본, 제주도, 남도, 남해, 전라도, 터키를 가보고 싶다고 했습니다. 가족과 휴양지로는 일본을 추천했습니다. 해외 여행지는 남아공, 멕시코, 브라질, 중국을 가보고 싶지 않다고 했습니다. SD2000569은 부산의 해운대, 동백섬, 광안리, 송도 케이블 카, 송도 조개구이, 캠핑을 추천했습니다. 가족 휴양지는 몰디브, 발리, 인도, 남아공, 인도네시아를 추천했습니다. 또 해외 여행지로는 유럽, 프랑스 파리를 추천했습니다.
Cosmos2: 두 화자는 이 대화에서 가보고 싶은 여행지, 가보고 싶은 해외 여행지, 가보고 싶은 휴양지 등에 대해 이야기했습니다. SD2000568은 국내 여행지로는 서울이나 자연경관을 볼 수 있는 한적한 곳을 추천했고, 해외 여행지로는 일본, 터키, 남도, 남해, 전라도, 제주도 등을 추천했습니다. 가족과 함께 가보고 싶은 휴양지는 일본이고, 가보고 싶은 해외 여행지는 터키라고 했습니다. 또한 가보고 싶은 휴양지로는 일본, 남도, 남해, 제주도, 전라도를 추천했으며, 가보고 싶은 해외 여행지는 터키라고 했습니다. SD2000569는 국내 여행지로는 해운대, 광안리, 송도 케이블카, 송도 조개 구, 캠핑을 추천했고, 가족과 함께 가보고 싶은 휴양지는 몰디브, 발리라고 했습니다. 또한 가보고 싶은 해외 여행지는 프랑스 파리라고 했습니다.
Cosmos25: 두 화자는 이 대화에서 가보고 싶은 여행지, 가보고 싶은 해외 여행지, 가보고 싶은 휴양지 등에 대해 이야기했습니다. SD2000568은 국내 여행지로는 서울이나 자연경관을 볼 수 있는 곳을 추천했고, 해외 여행지로는 일본, 터키, 일본을 추천했습니다. 가족과 함께 가보고 싶은 휴양지는 일본이고, 가보고 싶은 해외 여행지는 남도, 남해, 제

In [44]:
repeated_sentences

{' SD2000282는 SD2000283이 많이 먹는 것 같으니 차슈 덩밥을 먹고 싶어서 상무지구의 차슈 덩밥을 먹으러 가자고 말했습니다',
 ' SD2000282는 SD2000283이 많이 먹는 것 같으니 차슈 덮밥을 먹고 싶어서 상무지구의 차슈 덮밥을 먹으러 가자고 말했습니다',
 ' SD2000287은 중학교 1학년 때 동아리로 연극동아리에 가입하게 된 이유는 선생님께서 추천해주셨기 때문이라고 답했습니다',
 ' 단점으로는 감기와 추위가 심해 밖에서의 활동이 제한될 수 있다고 했습니다',
 ' 또 대전여중 앞쪽에 두부 두루치기가 있고, 성모병원 앞에도 두부 두루치기가 있다고 말했습니다',
 ' 또 자신이 선물을 선택할 때는 가성비보다는 자신이 좋아하는 것을 선물한다고 했습니다',
 ' 또 초등학교 때 가장 친했던 친구가 있었는지 물어봤으며, 중학교 때 연극동아리를 하면서 느낀 장점과 단점을 질문했습니다',
 ' 또한 혈액형은 성격과 관련이 없다는 것을 알고 있어서 혈액형에 의존하여 성격을 판단하지 않는다고 말했습니다',
 ' 여행 방식은 즉흥적인 여행을 선호한다고 말했습니다'}

In [49]:
# exact_repeated_sentences는 가징 첫 번째에 등장한 문장 1개로 통일
# + 이런 경우 comsos2에서 등장한 문장 중에서 cosmos25에 등장한 문장과 유사도가 낮은문장을 추가로 가져와 이어붙임
# 그 이유는 cosmos25에서 반복되는 문장 때문에 완전한 요약을 아직 못했을 수 있기 때문

def remove_repeated_sentences(data:json, exact_repeated_sentences:set, exact_idxes:set) -> None:
    """
    Remove exact repeated sentences from the data.

    Parameters:
    data (json): Data to remove repeated sentences from.
    exact_repeated_sentences (set): Set of exact repeated sentences.
    exact_idxes (set): Set of indexes of the exact repeated sentences.
    """
    for idx in sorted(exact_idxes, reverse=True):
        output = data[idx]['output']
        output_sentences = output.split('.')[:-1]
        for sentence in exact_repeated_sentences:
            if sentence in output_sentences:
                output_sentences.remove(sentence)
        data[idx]['output'] = '.'.join(output_sentences) + '.'

    with open('filtered_result.json', 'w') as file:
        json.dump(data, file, assure_ascii=False, indent=4)

In [117]:
def postprocessing(data:json):
    """
    postprocessing the cosmos_25.json file

    < Replace >
    [14] -> cosmos2
    [15] -> hypernova
    [26] -> hypernova
    [68] -> hypernova
    [98] -> hypernova + '\n\n' -> ' '
    [105] -> hypernova
    [219] -> hypernova
    [282] -> hypernova
    [365] -> cosmos2


    < Unify several into one >
    [130] 맨 마지막 3문장 삭제
    [166] 맨 마지막 4문장 삭제

    """


    with open(data, 'r') as f:
        data = json.load(f)

    # Replace
    data[14]['output'] = cosmos2['output'][14]
    data[15]['output'] = hypernova['output'][15]
    data[26]['output'] = hypernova['output'][26]
    data[68]['output'] = hypernova['output'][68]
    data[98]['output'] = hypernova['output'][98].replace('\n\n', ' ')
    data[105]['output'] = hypernova['output'][105]
    data[219]['output'] = hypernova['output'][219]
    data[282]['output'] = hypernova['output'][282]
    data[365]['output'] = cosmos2['output'][365]

    # Unify several into one
    data[130]['output'] = '.'.join(data[130]['output'].split('.')[:-4]) + '.'
    data[166]['output'] = '.'.join(data[166]['output'].split('.')[:-5]) + '.'


    with open('postprocessed_cosmos25.json', 'w') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

In [118]:
postprocessing('../results/cosmos_25.json')

In [119]:
# 후처리 이후
postprocessed_cosmos25 = make_dataframe('./postprocessed_cosmos25.json')
repeated_sentences, high_similarity_repeated_sentences, exact_idxes, high_idxes = find_repeated_sentence(postprocessed_cosmos25) # cosmosre12

100%|██████████| 408/408 [00:00<00:00, 859.14it/s]

Total number of exact repeated sentences found: 11
the number of exact repeated sentences found: 6
Total number of high similarity repeated sentences found: 22
the number of high similarity repeated sentences found: 21
Exact repeated sentences found in the following indexes: {328, 269, 23, 216, 221}
High similarity repeated sentences found in the following indexes: {129, 261, 137, 396, 14, 149, 169, 302, 310, 316, 328, 204, 206, 336, 349, 356, 103, 360, 111, 369, 125}


In [112]:
exact_idxes

{23, 216, 221, 269, 328}

In [113]:
compare_outputs(23)

Index: 23
Hypernova: 이 대화에서 화자 SD2000064와 SD2000065는 어버이날에 부모님께 주었던 선물과 선물 기준에 대해 이야기했습니다. SD2000064는 부모님께 어버이날과 어머니의 환갑을 기념하기 위해 영상을 만들어서 선물했습니다. 또 어버이날에는 학교나 꽃집에서 판매하는 카네이션을 선물하곤 했는데 이번에는 어머니의 환갑과 어버이날을 겸하여 영상을 선물했습니다. 화자 자신이 영상 편집을 잘 하는 편은 아니지만 어릴 때부터 부모님 결혼식부터 차곡차곡 모아두었던 사진들을 뒤져보며 영상을 만들었는데 감동을 많이 받아 눈물을 흘리기도 했다고 말했습니다. 화자는 선물을 주기 위해 어떤 기준을 세우는지 물어봤는데 화자 자신은 어렸을 때 부모님과 함께 친구 선물을 고를 때 받고 싶은 것을 줘야 한다고 말한 아버지의 말에 따라 자신이 받고 싶은 것들을 선물하는 편이라고 말했습니다. 또 화자는 가격 상관없이 자신이 좋아하는 것을 선물하는 편이지만 웃사 위인 사람들에게는 가격을 고려하여 선물을 주는 편이라고 말했습니다.

SD2000065는 부모님에게 어버이날로 초밥을 선물했습니다. 초밥은 예전 아르바이트했던 초밥집의 생선과 실장님의 정성을 다해 만든 초밥이기 때문에 믿고 갔다고 말했습니다. 또 가족끼리 좋은 곳을 가본 적이 없어서 초밥을 선택하게 된 것이라고 말했습니다. 화자에게 어버이 날 선물로 주었는지 물어보고 화자는 어버이 날과 어머니 환갑을 겸하여 카네이션과 영상을 선물했다고 말했습니다. 그리고 화자에게 선물을 주기 위한 기준이 있는지 물어봤고 화자는 어렸을 때 친구 선물 고르러 아버지와 함께 갔을 때 아버지가 받을 수 있는 것을 줘라 하셨기 때문에 자신이 좋아하는 것들을 선물한다고 말했습니다. 가격 상관없이 선물을 주지만 웃사인 사람에게는 가격을 고려해서 선물을 준다며 가격을 생각하게 되는 것 같다고 말했습니다.
Cosmos2: 두 화자는 이 대화에서 어버이날 선물과 선물 기준에 대해 이야기했습니다. SD2000064는 어버이날 선물로 

In [111]:
repeated_sentences

{' SD2000287은 중학교 1학년 때 동아리로 연극동아리에 가입하게 된 이유는 선생님께서 추천해주셨기 때문이라고 답했습니다',
 ' 단점으로는 감기와 추위가 심해 밖에서의 활동이 제한될 수 있다고 했습니다',
 ' 또 자신이 선물을 선택할 때는 가성비보다는 자신이 좋아하는 것을 선물한다고 했습니다',
 ' 또 초등학교 때 가장 친했던 친구가 있었는지 물어봤으며, 중학교 때 연극동아리를 하면서 느낀 장점과 단점을 질문했습니다',
 ' 또한 혈액형은 성격과 관련이 없다는 것을 알고 있어서 혈액형에 의존하여 성격을 판단하지 않는다고 말했습니다',
 ' 여행 방식은 즉흥적인 여행을 선호한다고 말했습니다'}

In [115]:
high_similarity_repeated_sentences

defaultdict(list,
            {' SD2000058은 가족 중에서 가장 건강에 염려스러운 것은 치매라고 말했습니다': [(14,
               ' SD2000058은 가족 중에서 가장 건강에 염려스러운 것은 치매라고 말했습니다',
               ' SD2000059는 가족 중에서 가장 건강에 염려스러운 것은 어머니의 고혈압이라고 말했습니다')],
             ' 가족과 함께 가는다면 단양을 추천하며, 친구들과 함께 가는다면 어디든 좋을 것 같다고 말했습니다': [(103,
               ' 가족과 함께 가는다면 단양을 추천하며, 친구들과 함께 가는다면 어디든 좋을 것 같다고 말했습니다',
               ' 가족과 함께 가는다면 어디든 좋을 것 같고, 친구들과 함께 가는다면 어디든 좋을 것 같다고 말했습니다')],
             ' SD2000160은 해외 여행지 추천과 국내 여행지 추천을 요청했고, 음식 맛 좋은 곳 추천을 요청했습니다': [(111,
               ' SD2000160은 해외 여행지 추천과 국내 여행지 추천을 요청했고, 음식 맛 좋은 곳 추천을 요청했습니다',
               ' SD2000161은 해외 여행지 추천과 국내 여행지 추천, 음식 맛 좋은 곳 추천, 관광지 추천을 했습니다'),
              (111,
               ' SD2000160은 해외 여행지 추천과 국내 여행지 추천을 요청했고, 음식 맛 좋은 곳 추천을 요청했습니다',
               ' SD2000160은 해외 여행지 추천을 요청했고, 국내 여행지 추천을 요청했습니다')],
             ' SD2002376은 상대방에게 최근에 연애를 했다고 말하며 상대방보다 연애를 빨리 식는 편이고, 상대방보다 연애를 더 오래하는 편이라고 말했습니다': [(125,
               ' SD2002376은 상대방에게

## test

In [84]:
import re

# 주어진 문장들
sentences = [
    "나는 사람이다 나는 사람이다 나는 사람이다",
    "나는 사람이다. 나는 사람이다. 나는 사람이다."
]

# 정규 표현식을 사용하여 중복된 부분 제거
def remove_duplicates(sentence):
    # 패턴을 통해 문장을 나누고, 중복을 제거
    pattern = r'(\b나는 사람이다\b[. ]*)'
    unique_sentence = ''.join([match.group(1) for i, match in enumerate(re.finditer(pattern, sentence)) if match.group(1) not in sentence[:match.start()]])
    return unique_sentence.strip()

# 각 문장에 대해 중복 제거 수행
results = [remove_duplicates(sentence) for sentence in sentences]

# 결과 출력
for result in results:
    print(result)

나는 사람이다
나는 사람이다.


## n-gram을 이용한 반복 문장 확인

In [54]:
from nltk.util import ngrams
from collections import Counter

def find_common_ngrams(df, n, threshold=2):
    """
    Find common n-grams in the 'output' column of the DataFrame.
    """

    sentences = df['output']

    def preprocess(sentence):
        return sentence.split()

    total_ngrams_dict = dict()
    cnt = 0

    for idx,sentence in enumerate(sentences):
        words = preprocess(sentence)
        ngrams_list = list(ngrams(words, n))
        ngrams_dict = Counter(ngrams_list)

        total_ngrams_dict[idx] = ngrams_dict

        if ngrams_dict.most_common(1)[0][1] > 2:
            print(f"Index: {idx}")
            print(f"Sentence: {sentence}")
            print(f"Most common {n}-gram: {ngrams_dict.most_common(1)}")
            print("\n")
            cnt+=1
    
    print(f"Total number of sentences with common {n}-grams which appear more than {threshold}: {cnt}")

    return total_ngrams_dict

In [60]:
total_ngrams_dict= find_common_ngrams(cosmos25, 6, 3)

Index: 14
Sentence: 두 화자는 이 대화에서 가족의 건강 상태와 자신의 건강 관리 방식에 대해 이야기했습니다. SD2000058은 가족 중에서 가장 건강에 염려스러운 것은 치매라고 말했습니다. 또한 자신은 현재 건강이 좋지만 치아가 나쁘기 때문에 치아 관리에 대해 질문했습니다. 또 가족 중에서 가장 건강에 염려스러운 것은 치매라고 말했고, 가족 중에서 가장 건강에 염려스러운 것은 치매라고 말했습니다. SD2000059는 가족 중에서 가장 건강에 염려스러운 것은 고혈압이라고 말했습니다. 또한 자신은 기립성 저혈압을 가지고 있어 운동을 통해 건강을 증진시키고 있다고 말했습니다. 그리고 가족 중에서 가장 건강에 염려스러운 것은 고혈압이라고 말했습니다.
Most common 6-gram: [(('가족', '중에서', '가장', '건강에', '염려스러운', '것은'), 5)]


Index: 68
Sentence: 두 화자는 이 대화에서 가족과의 관계와 환경에 대해 이야기했습니다. SD2000113은 자신이 어학연수에서 자유롭게 놀다가 귀국 후 가족과의 갈등이 발생했다고 말했습니다. 그리고 가족이 타인으로 대우받는다고 생각하며 가족과의 갈등을 해결하기 위해 타인을 기준으로 생각해보려고 한다며 가족과의 갈등을 해결하려는 방법을 설명했습니다. SD2000114는 가족이 타인으로 대우받는다고 생각하며 가족과의 갈등을 해결하기 위해 가족과의 갈등을 해결하려는 방법을 설명했습니다. 그리고 가족이 타인으로 대우받는다고 생각하며 가족과의 갈등을 해결하기 위해 가족과의 갈등을 해결하려는 방법을 설명했습니다.
Most common 6-gram: [(('가족이', '타인으로', '대우받는다고', '생각하며', '가족과의', '갈등을'), 3)]


Index: 130
Sentence: 두 화자는 이 대화에서 영화와 감독에 대해 이야기했습니다. SD2000174는 상을 받은 영화는 보지 않는 편이라고 말했습니다. 이창동 감독의 영화는 노트북으로 보는데 너무 어려워서 보지 않